# Non energetic emissions

This notebook calculates the ETLocal keys of the non-energetic emissions section.

## Import modules

In [1]:
import pandas as pd
from pathlib import Path
from IPython.display import Image, display
import importlib

import src.load_data_manager
import src.transform
import src.yaml_calculator
import src.helper
from src.load_data_manager import LoadDataManager
import config.yaml_to_diagram
from config.yaml_to_diagram import create_dependency_diagram
import src.co2_emissions_analysis

# Reload the modules to ensure latest version is used
importlib.reload(src.transform);
importlib.reload(src.yaml_calculator);
importlib.reload(src.helper);
importlib.reload(src.load_data_manager);
importlib.reload(config.yaml_to_diagram);

## General

Here we configure the notebook so that all contextual information is loaded and present for the other sections.


### Parameters

We first specify general parameters to be used throughout this notebook.

In [2]:
# Select the parent data set. Make sure to use the geo ID. It should be existing in the Dataset Manager.
parent = "nl"

# Specify the year and the reference year for the ETM
year = 2023
year_etm = 2019

# Specify the CSV-separator (presumably either "," or ";")
sep=","

# Specify where the dependency graphs are stored
dep_graph_folder = Path("config","etlocal_keys_final_demand_dependency_outputs", "output")

# Specify where the yaml files with calculation rules are stored
yaml_folder = Path("config","yaml_files_for_etlocal_key_calculation")

Here we define helper functions to be called for multiple cells throughout this specific notebook.

## Extract

In [3]:
data_manager = LoadDataManager()
df_template_local = data_manager.load_template(group = 'area')
df_template_local.head()


unit  \
geo_id group subgroup              key                                                         
GM1680 area  area_emission_factors file_carriers_natural_gas_co2_conversion_per_mj     kg/MJ   
                                   file_carriers_crude_oil_co2_conversion_per_mj       kg/MJ   
                                   file_carriers_coal_co2_conversion_per_mj            kg/MJ   
                                   file_carriers_lignite_co2_conversion_per_mj         kg/MJ   
                                   file_carriers_non_biogenic_waste_co2_conversion...  kg/MJ   

                                                                                        value  \
geo_id group subgroup              key                                                          
GM1680 area  area_emission_factors file_carriers_natural_gas_co2_conversion_per_mj     0.0565   
                                   file_carriers_crude_oil_co2_conversion_per_mj       0.0733   
                                   file_carriers_coal_co2_conversion_per_mj            0.0945   
                                   file_carriers_lignite_co2_conversion_per_mj         0.1012   
                                   file_carriers_non_biogenic_waste_co2_conversion...  0.1052   

                                                                                                                           commit  
geo_id group subgroup              key                                                                                             
GM1680 area  area_emission_factors file_carriers_natural_gas_co2_conversion_per_mj     Taken from the National 2023 dataset (ETM)  
                                   file_carriers_crude_oil_co2_conversion_per_mj       Taken from the National 2023 dataset (ETM)  
                                   file_carriers_coal_co2_conversion_per_mj            Taken from the National 2023 dataset (ETM)  
                                   file_carriers_lignite_co2_conversion_per_mj         Taken from the National 2023 dataset (ETM)  
                                   file_carriers_non_biogenic_waste_co2_conversion...  Taken from the National 2023 dataset (ETM)

In [4]:
input_data_test = pd.read_csv('data/pre_processing/non_energetic_emissions_output.csv', index_col=0)
input_data_test = input_data_test.sort_values(by='Gemeentenaam')
input_data_test.head()

,Gemeentenaam,energetic_emissions_other_ghg_agriculture,energetic_emissions_other_ghg_buildings,energetic_emissions_other_ghg_energy,energetic_emissions_other_ghg_households,energetic_emissions_other_ghg_industry,energetic_emissions_other_ghg_transport,non_energetic_emissions_other_ghg_agriculture_fermentation,non_energetic_emissions_other_ghg_agriculture_manure,non_energetic_emissions_other_ghg_agriculture_other,non_energetic_emissions_other_ghg_agriculture_soil_cultivation,non_energetic_emissions_other_ghg_chemical_industry,non_energetic_emissions_other_ghg_other_industry,non_energetic_emissions_other_ghg_waste_management,indirect_emissions_co2,non_energetic_emissions_co2_agriculture_manure,non_energetic_emissions_co2_agriculture_soil_cultivation,non_energetic_emissions_co2_chemical_industry,non_energetic_emissions_co2_other_industry,non_energetic_emissions_co2_waste_management
GemeenteCode,,,,,,,,,,,,,,,,,,,,
GM0518,'s-Gravenhage,0.003414,0.000073,0.012534,0.000622,0.000237,0.002789,0.000258,0.000003,0.0,0.000683,5.862303e-04,0.016849,0.008678,0.013578,0.000006,0.000002,0.001563,0.015881,0.0
GM0796,'s-Hertogenbosch,0.000291,0.000025,0.003718,0.000607,0.000198,0.002787,0.030386,0.007340,0.0,0.007943,3.284464e-03,0.009798,0.014988,0.003860,0.000123,0.000083,0.003167,0.004628,0.0
GM1680,Aa en Hunze,0.000182,0.000003,0.000518,0.000350,0.000523,0.000816,0.034970,0.003224,0.0,0.026058,0.000000e+00,0.000570,0.001100,0.000619,0.000433,0.000245,0.000000,0.000724,0.0
GM0358,Aalsmeer,0.006592,0.000004,0.000566,0.000201,0.000044,0.000449,0.000104,0.000034,0.0,0.000631,4.814983e-09,0.000993,0.000128,0.000798,0.000013,0.000008,0.000004,0.000933,0.0
GM0197,Aalten,0.000168,0.000004,0.000467,0.000275,0.000030,0.000244,0.044460,0.007427,0.0,0.011681,0.000000e+00,0.002110,0.001173,0.000656,0.000224,0.000132,0.000011,0.000768,0.0


In [ ]:
commit_messages = {
    "energetic_emissions_other_ghg_agriculture":
        "Energetic other greenhouse gas emissions from agriculture are calculated by converting nitrous oxide (N₂O) and methane (CH₄) emissions from agricultural fuel use into CO₂ equivalents, using IPCC conversion factors (Emissieregistratie, 2023).",

    "energetic_emissions_other_ghg_buildings":
        "Energetic other greenhouse gas emissions from buildings are calculated by converting N₂O and CH₄ emissions from heating in commercial and public buildings into CO₂ equivalents, using IPCC conversion factors (Emissieregistratie, 2023).",

    "energetic_emissions_other_ghg_energy":
        "Energetic other greenhouse gas emissions from the energy sector are calculated by converting N₂O and CH₄ emissions from oil and gas production and electricity generation into CO₂ equivalents, using IPCC conversion factors (Emissieregistratie, 2023).",

    "energetic_emissions_other_ghg_households":
        "Energetic other greenhouse gas emissions from households are calculated by converting nitrous oxide (N₂O) and methane (CH₄) emissions from residential combustion into CO₂ equivalents, using IPCC conversion factors (Emissieregistratie, 2023).",

    "energetic_emissions_other_ghg_industry":
        "Energetic other greenhouse gas emissions from industry are calculated by converting N₂O and CH₄ emissions from industrial fuel use into CO₂ equivalents, using IPCC conversion factors (Emissieregistratie, 2023).",

    "energetic_emissions_other_ghg_transport":
        "Energetic other greenhouse gas emissions from transport are calculated by converting N₂O and CH₄ emissions from vehicle exhaust into CO₂ equivalents, using IPCC conversion factors (Emissieregistratie, 2023).",

    "non_energetic_emissions_other_ghg_agriculture_fermentation":
        "Non-energetic other greenhouse gas emissions from agriculture (fermentation) are calculated by converting methane (CH₄) emissions from livestock digestion into CO₂ equivalents, using IPCC conversion factors (Emissieregistratie, 2023).",

    "non_energetic_emissions_other_ghg_agriculture_manure":
        "Non-energetic other greenhouse gas emissions from agriculture (manure) are calculated by converting N₂O and CH₄ emissions from livestock manure into CO₂ equivalents, using IPCC conversion factors (Emissieregistratie, 2023).",

    "non_energetic_emissions_other_ghg_agriculture_other":
        "Non-energetic other greenhouse gas emissions from agriculture are calculated by converting N₂O and CH₄ from various agricultural processes into CO₂ equivalents, using IPCC conversion factors (Emissieregistratie, 2023).",

    "non_energetic_emissions_other_ghg_agriculture_soil_cultivation":
        "Non-energetic other greenhouse gas emissions from agriculture (soil cultivation) are calculated by converting N₂O emissions from fertilized soils into CO₂ equivalents, using IPCC conversion factors (Emissieregistratie, 2023).",

    "non_energetic_emissions_other_ghg_chemical_industry":
        "Non-energetic other greenhouse gas emissions from the chemical industry are calculated by converting F-gases, N₂O and CH₄ from production processes into CO₂ equivalents, using IPCC conversion factors (Emissieregistratie, 2023).",

    "non_energetic_emissions_other_ghg_other_industry":
        "Non-energetic other greenhouse gas emissions from other industries are calculated by converting N₂O, CH₄ and F-gases from various industrial processes into CO₂ equivalents, using IPCC conversion factors (Emissieregistratie, 2023).",

    "non_energetic_emissions_other_ghg_waste_management":
        "Non-energetic other greenhouse gas emissions from waste management are calculated by converting N₂O and CH₄ emissions from waste treatment and landfills into CO₂ equivalents, using IPCC conversion factors (Emissieregistratie, 2023).",

    "indirect_emissions_co2":
        "Indirect CO₂ emissions are calculated from emissions that occur before or after direct energy use, based on activity data per sector (Emissieregistratie, 2023) and IPCC conversion factors.",

    "co2_emission_1990":    
        "Based on 1990 emissions data of the National dataset (ETM, 2023). This data is regionalised using the emission data from Emissieregistratie (2023). The sectors used to regionalise the emissions are 'Consumenten', 'Handel, Diensten en Overheid (HDO)', 'Verkeer en vervoer', 'Overige industrie', 'Chemische Industrie', 'Afvalverwijdering', 'Bouw', 'Raffinaderijen', 'Landbouw', 'Riolering en waterzuiveringsinstallaties'.",

    "non_energetic_emissions_co2_agriculture_manure": 
        "Non-energetic CO₂ emissions from agriculture (manure management) are calculated based on reported emissions from agricultural waste processing (Emissieregistratie, 2023).",

    "non_energetic_emissions_co2_waste_management":
        "Non-energetic CO₂ emissions from waste management are calculated based on emissions from incineration and composting processes (Emissieregistratie, 2023).",

    "non_energetic_emissions_co2_other_industry":
        "Non-energetic CO₂ emissions from other industry are calculated based on reported emissions from industrial processes not part of the chemical industry (Emissieregistratie, 2023).",

    "non_energetic_emissions_co2_chemical_industry":
        "Non-energetic CO₂ emissions from the chemical industry are calculated based on emissions from chemical production processes (Emissieregistratie, 2023).",

    "non_energetic_emissions_co2_agriculture_soil_cultivation":
        "Non-energetic CO₂ emissions from agriculture (soil cultivation) are calculated based on soil treatment practices and fertilizer use (Emissieregistratie, 2023)."
}


SyntaxError: invalid syntax (3675626101.py, line 47)

In [6]:
emissions_1990_path = Path("data", "raw", "ER_1990_emissions_data.xlsx")

In [9]:
emissions_1990_path = Path("data", "raw", "ER_1990_emissions_data.xlsx")
emissions_1990, analysis = src.co2_emissions_analysis.run_analysis_on_new_data(emissions_1990_path)
emissions_1990['GemeenteCode'] = 'GM' + emissions_1990['KM_code'].astype(str).str.zfill(4)
emissions_1990.set_index('GemeenteCode', inplace=True)
emissions_1990.rename(columns={'co2_1990_mt': 'co2_emission_1990'}, inplace=True)
input_data_test = input_data_test.join(emissions_1990[['co2_emission_1990']], how='left')

/opt/anaconda3/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Loaded 3769 emission records
Data covers: 343 unique areas
Sectors found: ['Bouw' 'Chemische Industrie' 'Energiesector' 'Verkeer en vervoer'
 'Riolering en waterzuiveringsinstallaties' 'Afvalverwijdering'
 'Drinkwatervoorziening' 'Overige industrie' 'Consumenten'
 'Handel, Diensten en Overheid (HDO)' 'Landbouw' 'Raffinaderijen']

Step 1: Creating lookup table from new data...
Available columns: ['Compartimentcode', 'Compartiment', 'Stofcode', 'Stof', 'Gebiedsindeling', 'Code_gebied', 'Gebied', 'Brontype', 'Sector', 'Subsector', 'Emissieoorzaak', 'Emissieoorzaak_code', 'Nic', 'Bedrijf', 'Straat', 'Huisnummer', 'Toevoeging', 'Postcode', 'Woonplaats', 'Xcoord', 'Ycoord', 'Sbi_code', 'Sbi_naam', 'Herkomst_code', 'Herkomst', 'Dataset', 'Jaar', 'Emissie', 'Eenheid']
Created lookup table with 3769 area-sector combinations

Step 2: Setting up municipality base...
Found 343 unique municipalities

Available sectors in new data: ['Bouw', 'Chemische Industrie', 'Energiesector', 'Verkeer en vervoer

old:

In [11]:
etlocal_keys = list(input_data_test.columns)[1:]
for i in range(len(etlocal_keys)):
    etlocal_key = etlocal_keys[i]
    df_pipeline = input_data_test[[etlocal_key]].copy()
    df_pipeline['commit'] = commit_messages.get(etlocal_key, "No commit messages found")
    df_template_local = src.helper.fill_template_with_etlocal_data(df_template_local,df_pipeline,etlocal_key=etlocal_keys[i])


In [12]:
etlocal_key = 'co2_emission_1990'
verification_results = src.helper.compare_template_with_verification(df_template_local, etlocal_key)
print("Verification completed.")


MUNICIPALITY COMPARISON FOR KEY: co2_emission_1990
  Template municipalities: 342
  Verification municipalities: 353
  Common municipalities: 338
  Missing in verification: 4
  Missing in template: 15

  MUNICIPALITIES IN TEMPLATE BUT NOT IN VERIFICATION:
     GM1980
     GM1982
     GM1991
     GM1992

  MUNICIPALITIES IN VERIFICATION BUT NOT IN TEMPLATE:
     GM0370
     GM0398
     GM0416
     GM0457
     GM0501
     GM0530
     GM0614
     GM0756
     GM0786
     GM0815
     GM0856
     GM1684
     GM1685
     GM1702
     Stap

 COMPARISON SUMMARY (338 municipalities):
  medium_diff: 274 municipalities (81.1%)
  small_diff: 62 municipalities (18.3%)
  large_diff: 2 municipalities (0.6%)

 VALUE STATISTICS:
  Template:     Min=0.00, Max=18.66, Mean=0.45
  Verification: Min=0.01, Max=19.21, Mean=0.46
  Abs diff:     Min=0.00, Max=0.55, Mean=0.01

 TOP 10 LARGEST DIFFERENCES:
  Municipality GM0599: Template=18.66, Verification=19.21, Diff=0.55 (-2.9%)
  Municipality GM1883: Template=

/Users/kaskranenburg/Code/etdataset/pipelines/src/helper.py:380: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  comparison["relative_diff"] = comparison["relative_diff"].replace(


In [ ]:
df_template_local

unit  \
geo_id group subgroup              key                                                    
GM0014 area  area                  analysis_year                                      #   
             area_emission_factors file_carriers_coal_co2_conversion_per_mj       kg/MJ   
                                   file_carriers_crude_oil_co2_conversion_per_mj  kg/MJ   
                                   file_carriers_diesel_co2_conversion_per_mj     kg/MJ   
                                   file_carriers_gasoline_co2_conversion_per_mj   kg/MJ   
...                                                                                 ...   
GM1992 area  area_infrastructure   mv_net_capacity_per_step                          MW   
                                   mv_net_costs_per_capacity_step                     €   
                                   mv_net_spare_capacity                              %   
                                   mv_net_total_costs_present                         €   
                                   offshore_net_costs_present                         €   

                                                                                   value  \
geo_id group subgroup              key                                                     
GM0014 area  area                  analysis_year                                     NaN   
             area_emission_factors file_carriers_coal_co2_conversion_per_mj       0.0945   
                                   file_carriers_crude_oil_co2_conversion_per_mj  0.0733   
                                   file_carriers_diesel_co2_conversion_per_mj     0.0743   
                                   file_carriers_gasoline_co2_conversion_per_mj   0.0720   
...                                                                                  ...   
GM1992 area  area_infrastructure   mv_net_capacity_per_step                       2.0000   
                                   mv_net_costs_per_capacity_step                 1.0000   
                                   mv_net_spare_capacity                          0.2500   
                                   mv_net_total_costs_present                        NaN   
                                   offshore_net_costs_present                        NaN   

                                                                                                                      commit  
geo_id group subgroup              key                                                                                        
GM0014 area  area                  analysis_year                                                                         nan  
             area_emission_factors file_carriers_coal_co2_conversion_per_mj       Taken from the National 2023 dataset (ETM)  
                                   file_carriers_crude_oil_co2_conversion_per_mj  Taken from the National 2023 dataset (ETM)  
                                   file_carriers_diesel_co2_conversion_per_mj     Taken from the National 2023 dataset (ETM)  
                                   file_carriers_gasoline_co2_conversion_per_mj   Taken from the National 2023 dataset (ETM)  
...                                                                                                                      ...  
GM1992 area  area_infrastructure   mv_net_capacity_per_step                       Taken from the National 2023 dataset (ETM)  
                                   mv_net_costs_per_capacity_step                 Taken from the National 2023 dataset (ETM)  
                                   mv_net_spare_capacity                          Taken from the National 2023 dataset (ETM)  
                                   mv_net_total_costs_present                                                            nan  
                                   offshore_net_costs_present                                                            nan  

[26334 rows x 3 columns]

## Export

Write the resulting filled template to a csv

In [ ]:
# Write the filled template to a CSV file
path_output = Path("data", "processed", f"etlocal_template_non_energetic_emissions_filled.csv")
df_template_local.to_csv(path_output, sep=sep)
